In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense

In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
replace __about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace hindi.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
batch_size=128
epochs=300
latent_dim=64
num_samples=900

In [ ]:
data_path='/content/hindi.txt'

# Vectorize the data.

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
print(target_texts)

['\tवाह!\n', '\tबचाओ!\n', '\tउछलो.\n', '\tकूदो.\n', '\tछलांग.\n', '\tनमस्ते।\n', '\tनमस्कार।\n', '\tवाह-वाह!\n', '\tचियर्स!\n', '\tसमझे कि नहीं?\n', '\tमैं ठीक हूँ।\n', '\tबहुत बढ़िया!\n', '\tअंदर आ जाओ।\n', '\tबाहर निकल जाओ!\n', '\tचले जाओ!\n', '\tख़ुदा हाफ़िज़।\n', '\tउत्तम!\n', '\tसही!\n', '\tआपका स्वागत है।\n', '\tस्वागतम्।\n', '\tमज़े करना।\n', '\tमौज करना।\n', '\tमज़े करो।\n', '\tमैं भूल गया।\n', '\tमैं भूल गई।\n', '\tमैं पैसे दूंगा।\n', '\tमैं ठीक हूँ।\n', '\tमेरा पेट भर गया है।\n', '\tचलो चलें!\n', '\tमुझे जवाब दो।\n', '\tपंछी उड़ते हैं।\n', '\tमाफ़ कीजिए।\n', '\tबहुत ख़ूब!\n', '\tमैं बेहोश हो गया।\n', '\tखेद की बात है, लेकिन वैसा ही है।\n', '\tमैं हँसा।\n', '\tमैं बोर हो रहा हूँ।\n', '\tमेरा दीवालिया हो चुका है।\n', '\tमैं थक गया हूँ।\n', '\tठंड हो रही है।\n', '\tशाबाश!\n', '\tकौन जाने?\n', '\tकिसको पता है?\n', '\tकिसे पता है?\n', '\tकिसे मालूम है?\n', '\tअद्भुत\n', '\tपंछी गाते हैं।\n', '\tअंदर आ जाओ।\n', '\tनिश्चित ही\n', '\tहिलो मत।\n', '\tआग जलाती है।\n', '\tउसका पीछा करो।

In [ ]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 900
Number of unique input tokens: 67
Number of unique output tokens: 77
Max sequence length for inputs: 25
Max sequence length for outputs: 60


In [ ]:
input_token_index=dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict(
[(char,i) for i, char in enumerate(target_characters)])

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn using adam optimizer & split 0.4
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model1 = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model1.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model1.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.4)

Epoch 1/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.8764 - loss: 0.4452 - val_accuracy: 0.7934 - val_loss: 0.7811
Epoch 2/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8785 - loss: 0.4339 - val_accuracy: 0.7918 - val_loss: 0.7875
Epoch 3/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8779 - loss: 0.4398 - val_accuracy: 0.7919 - val_loss: 0.7850
Epoch 4/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8754 - loss: 0.4430 - val_accuracy: 0.7912 - val_loss: 0.7905
Epoch 5/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8777 - loss: 0.4392 - val_accuracy: 0.7901 - val_loss: 0.7897
Epoch 6/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8820 - loss: 0.4244 - val_accuracy: 0.7918 - val_loss: 0.7895
Epoch 7/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8783 - loss: 0.4367 - val_accuracy: 0.7900 - val_loss: 0.7912
Epoch 8/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8785 - loss: 0.4385 - val_accuracy: 0.7903 - val_loss

In [ ]:
model1.save('hin2eng1.h5')

In [ ]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
         # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
len(input_texts)

900

In [ ]:
seen_sentences = set()

for seq_index in range(30):
    input_sentence = input_texts[seq_index]

    if input_sentence in seen_sentences:
        continue  # Skip if already decoded
    seen_sentences.add(input_sentence)

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('-')
    print('Input sentence:', input_sentence)
    print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
-
Input sentence: Wow!
Decoded sentence: शाबा!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
-
Input sentence: Help!
Decoded sentence: बचाओ!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
-
Input sentence: Jump.
Decoded sentence: उछलो.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━